## Requirements

### Web-Scraping
#### 1. Take the base wg-gesucht URL and scrape the shared flats listed on the website
#### 2. most important data points are price, room size and number of flat mates

### Data Analyis
#### 1. Correlation analysis of room price and room size. Hypothesis: positive correlation
#### 2. Correlation analysis of room price and number of flat mates. Hypothesis: negative correlation

### Import all neccessary libraries

In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

### Set base wg-gesucht url
### Add string interpolation to allow easy iteration through listing pages

In [25]:
pageNumber = 1
URL = f'https://www.wg-gesucht.de/wg-zimmer-in-Mannheim.85.0.0.{pageNumber}.html?pagination=1&pu='


### Iterate through listing pages and scrape the required data points
### Place scraped data into a dataframe/dictionary

In [34]:
pages = range(5)
df = pd.read_html(f'https://www.wg-gesucht.de/wg-zimmer-in-Mannheim.85.0.0.0.html?pagination=1&pu=')[0]

for i, page in enumerate(pages):
    df.append(pd.read_html(f'https://www.wg-gesucht.de/wg-zimmer-in-Mannheim.85.0.0.{i}.html?pagination=1&pu=')[0])
    print(df.head())


C:\Users\a794037\AppData\Local\Temp\ipykernel_19588\1335119623.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(pd.read_html(f'https://www.wg-gesucht.de/wg-zimmer-in-Mannheim.85.0.0.{i}.html?pagination=1&pu=')[0])


   Unnamed: 0  Bewohner     Eintrag Miete Größe              Stadtteil  \
0         NaN       NaN  12.04.2023  320€  15m²  Mannheim  Neckarstadt   
1         NaN       NaN  12.04.2023  590€  25m²    Mannheim  Lindenhof   
2         NaN       NaN  12.04.2023  260€  15m²  Mannheim  Neckarstadt   
3         NaN       NaN  12.04.2023  400€  11m²  Mannheim  Neckarstadt   
4         NaN       NaN  12.04.2023  389€  16m²  Mannheim  Neckarstadt   

      frei ab    frei bis  Unnamed: 8  
0  12.04.2023         NaN         NaN  
1  12.04.2023         NaN         NaN  
2  01.07.2023  30.09.2023         NaN  
3  15.04.2023         NaN         NaN  
4  01.06.2023  31.07.2023         NaN  


C:\Users\a794037\AppData\Local\Temp\ipykernel_19588\1335119623.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(pd.read_html(f'https://www.wg-gesucht.de/wg-zimmer-in-Mannheim.85.0.0.{i}.html?pagination=1&pu=')[0])


   Unnamed: 0  Bewohner     Eintrag Miete Größe              Stadtteil  \
0         NaN       NaN  12.04.2023  320€  15m²  Mannheim  Neckarstadt   
1         NaN       NaN  12.04.2023  590€  25m²    Mannheim  Lindenhof   
2         NaN       NaN  12.04.2023  260€  15m²  Mannheim  Neckarstadt   
3         NaN       NaN  12.04.2023  400€  11m²  Mannheim  Neckarstadt   
4         NaN       NaN  12.04.2023  389€  16m²  Mannheim  Neckarstadt   

      frei ab    frei bis  Unnamed: 8  
0  12.04.2023         NaN         NaN  
1  12.04.2023         NaN         NaN  
2  01.07.2023  30.09.2023         NaN  
3  15.04.2023         NaN         NaN  
4  01.06.2023  31.07.2023         NaN  


C:\Users\a794037\AppData\Local\Temp\ipykernel_19588\1335119623.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(pd.read_html(f'https://www.wg-gesucht.de/wg-zimmer-in-Mannheim.85.0.0.{i}.html?pagination=1&pu=')[0])


   Unnamed: 0  Bewohner     Eintrag Miete Größe              Stadtteil  \
0         NaN       NaN  12.04.2023  320€  15m²  Mannheim  Neckarstadt   
1         NaN       NaN  12.04.2023  590€  25m²    Mannheim  Lindenhof   
2         NaN       NaN  12.04.2023  260€  15m²  Mannheim  Neckarstadt   
3         NaN       NaN  12.04.2023  400€  11m²  Mannheim  Neckarstadt   
4         NaN       NaN  12.04.2023  389€  16m²  Mannheim  Neckarstadt   

      frei ab    frei bis  Unnamed: 8  
0  12.04.2023         NaN         NaN  
1  12.04.2023         NaN         NaN  
2  01.07.2023  30.09.2023         NaN  
3  15.04.2023         NaN         NaN  
4  01.06.2023  31.07.2023         NaN  


C:\Users\a794037\AppData\Local\Temp\ipykernel_19588\1335119623.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(pd.read_html(f'https://www.wg-gesucht.de/wg-zimmer-in-Mannheim.85.0.0.{i}.html?pagination=1&pu=')[0])


   Unnamed: 0  Bewohner     Eintrag Miete Größe              Stadtteil  \
0         NaN       NaN  12.04.2023  320€  15m²  Mannheim  Neckarstadt   
1         NaN       NaN  12.04.2023  590€  25m²    Mannheim  Lindenhof   
2         NaN       NaN  12.04.2023  260€  15m²  Mannheim  Neckarstadt   
3         NaN       NaN  12.04.2023  400€  11m²  Mannheim  Neckarstadt   
4         NaN       NaN  12.04.2023  389€  16m²  Mannheim  Neckarstadt   

      frei ab    frei bis  Unnamed: 8  
0  12.04.2023         NaN         NaN  
1  12.04.2023         NaN         NaN  
2  01.07.2023  30.09.2023         NaN  
3  15.04.2023         NaN         NaN  
4  01.06.2023  31.07.2023         NaN  
   Unnamed: 0  Bewohner     Eintrag Miete Größe              Stadtteil  \
0         NaN       NaN  12.04.2023  320€  15m²  Mannheim  Neckarstadt   
1         NaN       NaN  12.04.2023  590€  25m²    Mannheim  Lindenhof   
2         NaN       NaN  12.04.2023  260€  15m²  Mannheim  Neckarstadt   
3         NaN      

C:\Users\a794037\AppData\Local\Temp\ipykernel_19588\1335119623.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(pd.read_html(f'https://www.wg-gesucht.de/wg-zimmer-in-Mannheim.85.0.0.{i}.html?pagination=1&pu=')[0])


In [35]:
df.to_excel('wg-listing.xlsx')